<a href="https://colab.research.google.com/github/mzengin/randomtests/blob/main/evaluation_metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np

In [6]:
relevance = np.random.randint(low = 0, high=3, size=50)

In [7]:
relevance

array([1, 2, 1, 0, 1, 1, 0, 1, 2, 1, 0, 1, 2, 0, 2, 0, 1, 1, 0, 2, 0, 1,
       2, 1, 2, 2, 0, 0, 1, 0, 1, 0, 0, 0, 2, 0, 1, 1, 1, 0, 1, 1, 0, 1,
       0, 2, 1, 0, 2, 2])

In [30]:
# No negative relevance value
class EvaluationIR:
  def __init__(self):
    pass

  def precision(self, relevance, k):

    relevance = np.array(relevance)
    binned_relevance = np.bincount(relevance)
    return (len(relevance)-binned_relevance[0])/len(relevance)

  def recall(self, relevance, k):
    binned_relevance = np.bincount(relevance)
    rel_count = len(relevance) - binned_relevance[0]

    if rel_count == 0:
      return None

    binned_relevance_k = np.bincount(relevance[:k])
    ret_rel_cnt = k - binned_relevance_k[0]

    return ret_rel_cnt/rel_count


  def dcg(self, relevance, k):
    relevance = relevance[:k]
    dcg_score =  np.sum(relevance/ np.log2(np.arange(2, k+2)))
    return dcg_score

  def ndcg(self, relevance, k):
    if len(relevance) < k:
      relevance = relevance + [0 for _ in range(k - len(relevance))]

    relevance = np.array(relevance)
    sorted_relevance = np.sort(relevance)
    sorted_relevance = sorted_relevance[::-1]
    sorted_relevance = sorted_relevance[:k]

    idcg_score = self.dcg(sorted_relevance, k)
    dcg_score = self.dcg(relevance[:k], k)
    return dcg_score/idcg_score


In [34]:
eval = EvaluationIR()

In [35]:
eval.recall(relevance, 10)

0.25

In [73]:
class EvaluationML:
  def __init__(self):
    pass

  def true_positive(self, prediction, labels):
    tp = 0
    for pred, label in zip(prediction, labels):
      if pred == 1 and label == 1:
        tp += 1
    return tp

  def false_positive(self, prediction, labels):
    fp = 0
    for pred, label in zip(prediction, labels):
      if pred == 1 and label == 0:
        fp += 1
    return fp

  def true_negative(self, prediction, labels):
    tn = 0
    for pred, label in zip(prediction, labels):
      if pred == 0 and label == 0:
        tn += 1
    return tn

  def false_negative(self, prediction, labels):
    fn = 0
    for pred, label in zip(prediction, labels):
      if pred == 0 and label == 1:
        fn += 1
    return fn

  def precision(self, prediction, labels):
    tp = self.true_positive(prediction, labels)
    fp = self.false_positive(prediction, labels)

    return tp/(tp + fp + 1e-5)

  def recall(self, prediction, labels):
    tp = self.true_positive(prediction, labels)
    fn = self.false_negative(prediction, labels)
    return tp/(tp + fn + 1e-5)

  def get_confusion_matrix(self, prediction, labels):
    tp = self.true_positive(prediction, labels)
    fp = self.false_positive(prediction, labels)
    tn = self.true_negative(prediction, labels)
    fn = self.false_negative(prediction, labels)

    #            Actual 0,   Actual 1
    # Predict 0  TN          FN
    # Predict 1  FP          TP
    return {'tn': tn, 'fn': fn, 'fp': fp, 'tp': tp}

In [74]:
evalml = EvaluationML()

In [46]:
v1= np.array([1,1,0,1])

In [76]:
evalml.recall(np.array([1,0,1,1]),np.array([1,1,1,1]))

0.7499981250046875